# Clustering Neighborhoods in Toronto

Let's start by installing the necessary packages and libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191111203411-0002
KERNEL_ID = de9e9609-a041-436f-a07b-d5c5ceb9adfc
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Now we will load brute data concerning the Neighbourhood in Toronto. We will store this in a Datafarme called all_nb

In [2]:
#import the table using the pandas read_html method
#take the first returned object as teh brute Neighbourghood 
pd_list = pd.read_html ("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

all_nb = pd_list[0]
all_nb.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Check the shape of teh initial Dataframe. Apparently, we have 233 lines

In [3]:
all_nb.shape

(288, 3)

Drop the lines whith 'Not assigned' borough. There are 211 lines remaining

In [5]:
all_nb.drop(all_nb.loc[all_nb['Borough']=="Not assigned"].index, axis=0, inplace=True)
all_nb.shape

(211, 3)

Set Neighbourhood to Borough for theose lines where Neighbourhood is 'Not Assigned'

In [7]:
all_nb.loc[all_nb['Neighbourhood']=="Not assigned", 'Neighbourhood'] = all_nb.loc[all_nb['Neighbourhood']=="Not assigned"]['Borough']

Sort the remaining rows ans reset the index

In [8]:
all_nb.sort_values(by=['Postcode', 'Borough', 'Neighbourhood'], inplace=True)
all_nb.reset_index(inplace=True, drop=True)
all_nb

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Rouge Hill
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


How many different Neighbourhoods do we have per Borough?

In [10]:
all_nb.groupby(['Postcode', 'Borough']).count()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,2
M1C,Scarborough,3
M1E,Scarborough,3
M1G,Scarborough,1
M1H,Scarborough,1
M1J,Scarborough,1
M1K,Scarborough,3
M1L,Scarborough,3
M1M,Scarborough,3


Conacatenate the names of the Neighbourhoods which belong to the the same Borough. 

The final dataframe si called **nbhood**

In [11]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
nbhood = pd.DataFrame(columns=column_names)
nbhood = nbhood.append(all_nb.loc[0])

j = 0
for i in range(1, all_nb.shape[0]):
    Postcode = all_nb.iloc[i]['Postcode']
    Borough = all_nb.iloc[i]['Borough']
    Neighbourhood = all_nb.iloc[i]['Neighbourhood']
    if nbhood.iloc[j]['Postcode'] == Postcode and nbhood.iloc[j]['Borough'] == Borough:
        nbhood.iloc[j]['Neighbourhood'] = nbhood.iloc[j]['Neighbourhood'] + ', ' + Neighbourhood
    else:
        nbhood = nbhood.append([{'Postcode':Postcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood}], ignore_index=True)
        j = j + 1
   
nbhood

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


How many lines we have now?

In [13]:
nbhood.shape

(103, 3)